### Постановка задачи

Необходимо предсказать биологический ответ молекул (столбец 'Activity') по их химическому составу (столбцы D1-D1776).

Данные представлены в формате CSV. Каждая строка представляет молекулу.

Первый столбец Activity содержит экспериментальные данные, описывающие фактический биологический ответ [0, 1];
Остальные столбцы D1-D1776 представляют собой молекулярные дескрипторы — это вычисляемые свойства, которые могут фиксировать некоторые характеристики молекулы, например размер, форму или состав элементов.
Предварительная обработка не требуется, данные уже закодированы и нормализованы.

В качестве метрики будем использовать F1-score.

Необходимо обучить две модели: логистическую регрессию и случайный лес. Далее нужно сделать подбор гиперпараметров с помощью базовых и продвинутых методов оптимизации. Важно использовать все четыре метода (GridSeachCV, RandomizedSearchCV, Hyperopt, Optuna) хотя бы по разу, максимальное количество итераций не должно превышать 50.

In [1]:
#импорт библиотек
import numpy as np #для матричных вычислений
import pandas as pd #для анализа и предобработки данных
import matplotlib.pyplot as plt #для визуализации
import seaborn as sns #для визуализации

from sklearn import linear_model #линейные моделиё
from sklearn import tree #деревья решений
from sklearn import ensemble #ансамбли
from sklearn import metrics #метрики
from sklearn import preprocessing #предобработка
from sklearn.model_selection import train_test_split #сплитование выборки
from sklearn.model_selection import cross_val_score #Кросс валидация

# Импорт оптимизаторов параметров
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

import hyperopt
from hyperopt import hp, fmin, tpe, Trials

import optuna

%matplotlib inline
plt.style.use('seaborn')

In [2]:
data = pd.read_csv('data/_train_sem09.csv')
data.head()

,Activity,D1,D2,D3,D4,D5,D6,D7,D8,D9,...,D1767,D1768,D1769,D1770,D1771,D1772,D1773,D1774,D1775,D1776
0,1,0.000000,0.497009,0.10,0.0,0.132956,0.678031,0.273166,0.585445,0.743663,...,0,0,0,0,0,0,0,0,0,0
1,1,0.366667,0.606291,0.05,0.0,0.111209,0.803455,0.106105,0.411754,0.836582,...,1,1,1,1,0,1,0,0,1,0
2,1,0.033300,0.480124,0.00,0.0,0.209791,0.610350,0.356453,0.517720,0.679051,...,0,0,0,0,0,0,0,0,0,0
3,1,0.000000,0.538825,0.00,0.5,0.196344,0.724230,0.235606,0.288764,0.805110,...,0,0,0,0,0,0,0,0,0,0
4,0,0.100000,0.517794,0.00,0.0,0.494734,0.781422,0.154361,0.303809,0.812646,...,0,0,0,0,0,0,0,0,0,0


In [3]:
X = data.drop(['Activity'], axis=1)
y = data['Activity']

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = 0.3)

# Мое решение

Сначала получим значения меток F1-score для логистической регресии и случайного леса без использования методов оптимизации, чтобы иметь в дальнейшем базу для сравнения

### Логистическая регрессия (без методов оптимизации)

In [8]:
#Создаем объект класса логистическая регрессия
log_reg = linear_model.LogisticRegression(random_state=42, max_iter = 50)
#Обучаем модель
log_reg.fit(X_train, y_train)

y_test_pred = log_reg.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

f1_score на тестовом наборе: 0.78


C:\Users\ASUS\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


### Случайный лес (без методов оптимизации)

In [13]:
#Создаем объект класса случайный лес
rf = ensemble.RandomForestClassifier(random_state=42)
#Обучаем модель
rf.fit(X_train, y_train)

y_test_pred = rf.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

f1_score на тестовом наборе: 0.82


Далее последовательно используются методы оптимизации

# Метод GridSeachCV

### Логистическая регрессия, метод GridSeachCV

In [10]:
param_grid = [
    {'penalty' : ['l2', 'none'], # тип регуляризации
    'solver' : ['newton-cg', 'lbfgs', 'sag'], # алгоритм оптимизации
    'C' : [0.01, 0.1, 0.3, 0.5, 0.7, 0.9, 1]}, # уровень силы регурялизации
    # другой набор гиперпараметров
    {'penalty': ['l1', 'l2'] ,
    'solver': ['liblinear', 'saga'],
    'C': [0.01, 0.1, 0.3, 0.5, 0.7, 0.9, 1]}
]

grid_search_LR = GridSearchCV(
    estimator = linear_model.LogisticRegression(random_state= 42, max_iter = 50),
    param_grid = param_grid,
    cv = 5,
    n_jobs = -1,
    scoring='f1'
)
%time grid_search_LR.fit(X_train, y_train)

y_test_pred = grid_search_LR.predict(X_test)

print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(grid_search_LR.best_params_))

CPU times: total: 7.5 s
Wall time: 8min 38s
f1_score на тестовом наборе: 0.79
Наилучшие значения гиперпараметров: {'C': 0.1, 'penalty': 'l2', 'solver': 'saga'}


C:\Users\ASUS\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


### Случайный лес, метод GridSeachCV

In [34]:
param_distributions = {'n_estimators': list(range(80, 200, 30)),
              'min_samples_leaf': list(np.linspace(5, 25, 5, dtype=int)),
              'max_depth': list(np.linspace(1, 40, 5, dtype=int))
              }
            
random_search_forest = RandomizedSearchCV(
    estimator=ensemble.RandomForestClassifier(random_state=42), 
    param_distributions=param_distributions, 
    cv=5,
    n_iter = 50, 
    n_jobs = -1,
    scoring = 'f1'
)  
%time random_search_forest.fit(X_train, y_train) 

y_test_pred = random_search_forest.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(random_search_forest.best_params_))

CPU times: total: 3.22 s
Wall time: 57.7 s
f1_score на тестовом наборе: 0.82
Наилучшие значения гиперпараметров: {'n_estimators': 140, 'min_samples_leaf': 5, 'max_depth': 40}


# Метод RandomizedSeachCV

### Логистическая регрессия, метод RandomizedSeachCV

In [21]:
param_distributions = [
                        {'penalty': ['l2', 'none'] , # тип регуляризации
                         'solver' : ['newton-cg', 'lbfgs', 'sag'], # алгоритм оптимизации
                          'C': list(np.linspace(0.01, 1, 10, dtype=float))
                        },
                        {'penalty': ['l1', 'l2'] ,
                         'solver': ['liblinear', 'saga'],
                          'C': list(np.linspace(0.01, 1, 10, dtype=float))
                        }]
random_search_LR = RandomizedSearchCV(
    estimator=linear_model.LogisticRegression(random_state=42, max_iter= 50), 
    param_distributions=param_distributions, 
    cv=5,
    n_iter = 20, 
    n_jobs = -1,
    scoring='f1'
)  
%time random_search_LR.fit(X_train, y_train) 

y_test_pred = random_search_LR.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(random_search_LR.best_params_))

CPU times: total: 3.25 s
Wall time: 50.5 s
f1_score на тестовом наборе: 0.79
Наилучшие значения гиперпараметров: {'solver': 'saga', 'penalty': 'l2', 'C': 0.12}


C:\Users\ASUS\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


### Случайный лес, метод RandomizedSeachCV

In [38]:
param_distributions = {'n_estimators': list(range(80, 200, 30)),
              'min_samples_leaf': list(np.linspace(5, 25, 5, dtype=int)),
              'max_depth': list(np.linspace(1, 40, 5, dtype=int))
              }
            
random_search_RF = RandomizedSearchCV(
    estimator=ensemble.RandomForestClassifier(random_state=42), 
    param_distributions=param_distributions, 
    cv=5,
    n_iter = 50, 
    n_jobs = -1,
    scoring= 'f1'
)  
%time random_search_RF.fit(X_train, y_train) 

y_test_pred = random_search_RF.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(random_search_RF.best_params_))

CPU times: total: 3.62 s
Wall time: 58.1 s
f1_score на тестовом наборе: 0.82
Наилучшие значения гиперпараметров: {'n_estimators': 170, 'min_samples_leaf': 5, 'max_depth': 30}


# Метод Hyperopt

In [39]:
# Уточняем текущую версию HyperOpt
print("Версия Hyperopt : {}".format(hyperopt.__version__))

Версия Hyperopt : 0.2.5


### Логистическая регрессия, метод HyperOpt

In [68]:
space ={
    'penalty' : hp.choice(label='penalty', options= ['l1', 'l2']), # тип регуляризации
    'solver' : hp.choice(label = 'solver', options= ['liblinear', 'saga']), # алгоритм оптимизации
    'C' : hp.loguniform(label='C', low=-2*np.log(10), high=2*np.log(10)), # уровень силы регурялизации
    'max_iter': hp.choice('max_iter', [50])}

In [69]:
random_state = 42
def hyperopt_logr(params, cv=5, X=X_train, y=y_train, random_state=random_state):
    params = {'penalty': params['penalty'],
              'solver': params['solver'],
              'C': params['C'],
              'max_iter': params['max_iter']
              }
    model_HO = linear_model.LogisticRegression(**params, random_state=random_state)

    # обучаем модель
    model_HO.fit(X, y)
    score = cross_val_score(model_HO, X, y, cv=cv, scoring="f1", n_jobs=-1).mean()

    # метрику необходимо минимизировать, поэтому ставим знак минус
    return -score

In [70]:
%%time
# начинаем подбор гиперпараметров
trials = Trials() # используется для логирования результатов

best=fmin(hyperopt_logr, # наша функция 
          space=space, # пространство гиперпараметров
          algo=tpe.suggest, # алгоритм оптимизации, установлен по умолчанию, задавать необязательно
          max_evals=20, # максимальное количество итераций
          trials=trials, # логирование результатов
          # rstate = np.random.default_rng(42) # для версии 0.2.7
          rstate=np.random.RandomState(42) # (random_state)# фиксируем для повторяемости результата
         )
print("Наилучшие значения гиперпараметров {}".format(best))

  0%|                                                                           | 0/20 [00:00<?, ?trial/s, best loss=?]

C:\Users\ASUS\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(



 20%|█████████▌                                      | 4/20 [00:44<03:15, 12.21s/trial, best loss: -0.7644504520739478]

C:\Users\ASUS\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(



 25%|████████████                                    | 5/20 [00:51<02:35, 10.39s/trial, best loss: -0.7644504520739478]

...

 80%|█████████████████████████████████████▌         | 16/20 [02:43<00:27,  6.99s/trial, best loss: -0.7773040943091213]

C:\Users\ASUS\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(



 90%|██████████████████████████████████████████▎    | 18/20 [02:58<00:14,  7.34s/trial, best loss: -0.7773040943091213]

C:\Users\ASUS\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(



100%|███████████████████████████████████████████████| 20/20 [03:16<00:00,  9.81s/trial, best loss: -0.7773040943091213]
Наилучшие значения гиперпараметров {'C': 0.1694465206213903, 'max_iter': 0, 'penalty': 1, 'solver': 1}
CPU times: total: 1min 26s
Wall time: 3min 16s


Поскольку данная версия hyperopt в качестве индексов массива выдает для логистической регрессии только индексы массива, то воспользуемся space_eval, чтобы обращаться к элементам массива напрямую

In [71]:
from hyperopt import space_eval
hyperparams = space_eval(space, best)
hyperparams

{'C': 0.1694465206213903, 'max_iter': 50, 'penalty': 'l2', 'solver': 'saga'}

In [72]:
# рассчитаем точность для тестовой выборки
model_HO = linear_model.LogisticRegression(
    random_state=random_state,
    penalty = hyperparams['penalty'],
    solver = hyperparams['solver'],
    C = hyperparams['C'],
    max_iter = hyperparams['max_iter']
    )
model_HO.fit(X, y)

y_test_pred = model_HO.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

f1_score на тестовом наборе: 0.87


C:\Users\ASUS\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


### Случайный лес, метод HyperOpt

In [128]:
# зададим пространство поиска гиперпараметров
space={'n_estimators': hp.quniform('n_estimators', 80, 200, 20),
       'max_depth' : hp.quniform('max_depth', 1, 30, 1),
       'min_samples_leaf': hp.quniform('min_samples_leaf', 5, 25, 1)
      }

In [129]:
def hyperopt_rf(params, cv=5, X=X_train, y=y_train, random_state=random_state):
    
    params = {'n_estimators': int(params['n_estimators']), 
              'max_depth': int(params['max_depth']), 
             'min_samples_leaf': int(params['min_samples_leaf'])
    }
    # используем эту комбинацию для построения модели   
    model_HO_RF = ensemble.RandomForestClassifier(**params, random_state=random_state)

    # обучаем модель
    model_HO_RF.fit(X, y)
    score = cross_val_score(model_HO_RF, X, y, cv=cv, scoring="f1", n_jobs=-1).mean()
    # score = metrics.f1_score(y, model_HO_RF.predict(X))
    return -score  

In [130]:
%%time
# начинаем подбор гиперпараметров
trials = Trials() # используется для логирования результатов

best=fmin(hyperopt_rf, # наша функция 
          space=space, # пространство гиперпараметров
          algo=tpe.suggest, # алгоритм оптимизации, установлен по умолчанию, задавать необязательно
          max_evals=20, # максимальное количество итераций
          trials=trials, # логирование результатов
          # rstate = np.random.default_rng(42) # для версии 0.2.7
          rstate=np.random.RandomState(42) # (random_state)# для версии 0.2.5, фиксируем для повторяемости результата
         )
print("Наилучшие значения гиперпараметров {}".format(best))

100%|███████████████████████████████████████████████| 20/20 [01:08<00:00,  3.41s/trial, best loss: -0.8006240707439053]
Наилучшие значения гиперпараметров {'max_depth': 26.0, 'min_samples_leaf': 5.0, 'n_estimators': 140.0}
CPU times: total: 28.6 s
Wall time: 1min 8s


In [131]:
# рассчитаем точность для тестовой выборки
model_HO_RF = ensemble.RandomForestClassifier(
    random_state=random_state, 
    n_estimators=int(best['n_estimators']),
    max_depth=int(best['max_depth']),
    min_samples_leaf=int(best['min_samples_leaf'])
)

model_HO_RF.fit(X, y)

y_test_pred = model_HO_RF.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

f1_score на тестовом наборе: 0.95


# Метод Optuna

In [5]:
print("Версия Optuna: {}".format(optuna.__version__))

Версия Optuna: 3.0.2


### Логистическая регрессия, метод Optuna 

In [98]:
random_state=42
def optuna_lr(trial):
    penalty = trial.suggest_categorical('penalty', ['l1', 'l2'])
    solver = trial.suggest_categorical('solver', ['liblinear', 'saga'])
    C = trial.suggest_float(name='C', low=0.01, high=1.01, step = 0.05)
   
    ## Создаем модель
    model_Op_LR = LogisticRegression(
                                    penalty=penalty,
                                    solver=solver,
                                    C=C,
                                    random_state=random_state
                              )
    ## Обучаем модель
    model_Op_LR.fit(X_train, y_train)
    score = cross_val_score(model_Op_LR, X_train, y_train, cv=5, scoring="f1", n_jobs=-1).mean()
    
    return score

In [99]:
%%time
study = optuna.create_study(study_name="LogisticRegression", direction="maximize")
study.optimize(optuna_lr, n_trials=20)

[I 2022-10-08 16:18:19,963] A new study created in memory with name: LogisticRegression
C:\Users\ASUS\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
[I 2022-10-08 16:18:29,664] Trial 0 finished with value: 0.7652400393220528 and parameters: {'penalty': 'l2', 'solver': 'saga', 'C': 0.51}. Best is trial 0 with value: 0.7652400393220528.
[I 2022-10-08 16:18:31,616] Trial 1 finished with value: 0.7642270161739722 and parameters: {'penalty': 'l2', 'solver': 'liblinear', 'C': 0.46}. Best is trial 0 with value: 0.7652400393220528.
[I 2022-10-08 16:18:32,649] Trial 2 finished with value: 0.7682614041849851 and parameters: {'penalty': 'l2', 'solver': 'liblinear', 'C': 0.01}. Best is trial 2 with value: 0.7682614041849851.
[I 2022-10-08 16:18:33,388] Trial 3 finished with value: 0.74701333922839 and parameters: {'penalty': 'l1', 'solver': 'liblinear', 'C': 0.01}. Best is trial 2 wi

CPU times: total: 1min 1s
Wall time: 2min 21s


In [100]:
# выводим наилучшие значения гиперпараметров
print("Наилучшие значения гиперпараметров {}".format(study.best_params))

Наилучшие значения гиперпараметров {'penalty': 'l1', 'solver': 'liblinear', 'C': 0.41000000000000003}


In [101]:
model_Op_LR = LogisticRegression(**study.best_params, random_state=random_state,)

model_Op_LR.fit(X_train, y_train)

y_test_pred = model_Op_LR.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

f1_score на тестовом наборе: 0.79


### Случайный лес, метод Optuna

In [6]:
random_state = 42
def optuna_rf(trial):
  # задаем пространства поиска гиперпараметров
  n_estimators = trial.suggest_int('n_estimators', 100, 400, 1)  #100, 200, 1
  max_depth = trial.suggest_int('max_depth', 5, 30, 1)     # 10, 30, 1
  min_samples_leaf = trial.suggest_int('min_samples_leaf', 2, 12, 1)

  # создаем модель
  model_Op_rf = ensemble.RandomForestClassifier(n_estimators=n_estimators,
                                          max_depth=max_depth,
                                          min_samples_leaf=min_samples_leaf,
                                          random_state=random_state)
  # обучаем модель
  model_Op_rf.fit(X_train, y_train)
  score = cross_val_score(model_Op_rf, X_train, y_train, cv = 5, scoring = 'f1', n_jobs= -1).mean()
    
  return score

In [10]:
%%time
# cоздаем объект исследования
# можем напрямую указать, что нам необходимо максимизировать метрику direction="maximize"
study = optuna.create_study(study_name="RandomForestClassifier", direction="maximize")
# ищем лучшую комбинацию гиперпараметров n_trials раз
study.optimize(optuna_rf, n_trials=50)

[I 2022-10-15 18:11:18,999] A new study created in memory with name: RandomForestClassifier
[I 2022-10-15 18:11:26,531] Trial 0 finished with value: 0.7969494519959952 and parameters: {'n_estimators': 138, 'max_depth': 27, 'min_samples_leaf': 8}. Best is trial 0 with value: 0.7969494519959952.
[I 2022-10-15 18:11:37,032] Trial 1 finished with value: 0.7965911367526675 and parameters: {'n_estimators': 376, 'max_depth': 20, 'min_samples_leaf': 9}. Best is trial 0 with value: 0.7969494519959952.
[I 2022-10-15 18:11:47,480] Trial 2 finished with value: 0.7948452397552549 and parameters: {'n_estimators': 397, 'max_depth': 21, 'min_samples_leaf': 7}. Best is trial 0 with value: 0.7969494519959952.
[I 2022-10-15 18:11:57,362] Trial 3 finished with value: 0.8051868571215997 and parameters: {'n_estimators': 359, 'max_depth': 12, 'min_samples_leaf': 3}. Best is trial 3 with value: 0.8051868571215997.
[I 2022-10-15 18:12:02,546] Trial 4 finished with value: 0.7861555518725513 and parameters: {'n_

[I 2022-10-15 18:17:02,969] Trial 38 finished with value: 0.8060674705411547 and parameters: {'n_estimators': 365, 'max_depth': 29, 'min_samples_leaf': 3}. Best is trial 21 with value: 0.8125296713388959.
[I 2022-10-15 18:17:11,013] Trial 39 finished with value: 0.7886982231538716 and parameters: {'n_estimators': 329, 'max_depth': 26, 'min_samples_leaf': 10}. Best is trial 21 with value: 0.8125296713388959.
[I 2022-10-15 18:17:18,674] Trial 40 finished with value: 0.7951539935868285 and parameters: {'n_estimators': 301, 'max_depth': 21, 'min_samples_leaf': 8}. Best is trial 21 with value: 0.8125296713388959.
[I 2022-10-15 18:17:30,439] Trial 41 finished with value: 0.810847641628037 and parameters: {'n_estimators': 351, 'max_depth': 28, 'min_samples_leaf': 2}. Best is trial 21 with value: 0.8125296713388959.
[I 2022-10-15 18:17:41,604] Trial 42 finished with value: 0.8109449144248797 and parameters: {'n_estimators': 330, 'max_depth': 28, 'min_samples_leaf': 2}. Best is trial 21 with va

CPU times: total: 3min 21s
Wall time: 7min 40s


In [11]:
# выводим наилучшие значения гиперпараметров
print("Наилучшие значения гиперпараметров {}".format(study.best_params))

Наилучшие значения гиперпараметров {'n_estimators': 335, 'max_depth': 30, 'min_samples_leaf': 2}


In [12]:
# рассчитаем точность для тестовой выборки
model_Op_rf = ensemble.RandomForestClassifier(**study.best_params,random_state=random_state, )
model_Op_rf.fit(X_train, y_train)
y_test_pred = model_Op_rf.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

f1_score на тестовом наборе: 0.82


### Некоторые выводы

И для случайного леса, и для логистической регрессии наилучший результат показал метод HyperOpt. Возможно, его алгоритмы оптимальны в данном случае, но отсутствие на него нормальной документации меня (думаю, как и многих) останавливает от широкого использования. Та же Optuna имеет более понятный интерфейс и даже более обширный пакет документации, хотя и является более молодым продуктом на рынке